In [24]:
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import sys
%matplotlib inline
import os
import re
import random
from itertools import product
import pandas as pd
import pickle as pkl
from funcy import flatten
from nltk import everygrams, ngrams
import io
# Let's write the tokenization function 
# import spacy
import string
#stopwords list
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')



[nltk_data] Downloading package stopwords to /Users/Ji/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Ji/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Ji/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [35]:

#load each txt and return a list of data, a list of target
def load_txt(filepath):
    data = {}
    for txt in os.listdir(filepath):
        if not txt.startswith('.'):
            txt_id = txt.split(".")[0]
            txt_path = filepath + '/' + txt
            with open(txt_path, 'r', encoding='utf8', errors='ignore') as f:
    #         with io.open(txt_path, 'r', encoding='windows-1252') as f:
    #         with open(txt_path, 'r') as f:
                txtfile = f.read() #f.read() can only be read by once
    #             if len(txtfile) != 0: #remove empty txt
    #                 new_text=re.sub('<br />','',txtfile) #remove '<br />'
                data[txt_id] = txtfile
    #             data.append(txtfile)   
    return data

# filepath = sys.argv[1]
# filepath = '../data/input/files/text'
filepath = '../../train_test/files/text/'

test = load_txt(filepath)

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

wnl = WordNetLemmatizer()

# Load English tokenizer, tagger, parser, NER and word vectors
# tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation

# lowercase and remove punctuation, remove stopwords, stemming words
# only lemmatize noun

def tokenize(sent):
#     if (token.text not in punctuations)
    tokens = word_tokenize(sent)
    tokens_lower = [token.lower() for token in tokens if (token not in punctuations) 
                    and (token not in stop_words)]
    tokens_lemma = [wnl.lemmatize(word) for word in tokens_lower]

    
    return tokens_lemma

def tokenize_dataset(dataset):
    token_dataset_1gram = {}
    token_dataset_2gram = {}
    token_dataset_3gram = {}
    
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later

    for index in dataset:
        tokens = tokenize(dataset[index])
        #2grams
        
        tokens_2gram = list(ngrams(tokens,2))
        token_dataset_2gram[index]=tokens_2gram
#         token_dataset_2gram.append(tokens_2gram)
        
        #3grams
        tokens_3gram = list(ngrams(tokens,3))
#         token_dataset_3gram.append(tokens_3gram)
        token_dataset_3gram[index]=tokens_3gram 
        

    return (token_dataset_2gram,token_dataset_3gram)  #token_dataset_1gram,

# test set tokens

print ("Tokenizing train data")
(train_data_tokens_2,train_data_tokens_3) = tokenize_dataset(test)





Tokenizing train data


In [36]:
import json
import os
from pprint import pprint


with open('../../train_test/sage_research_methods.json') as f:
    methods = json.load(f)

pprint(methods['@graph'][0]['skos:prefLabel']['@value'])

'Population parameters'


In [37]:
methods

{'@context': {'luxid': 'http://www.temis.com/luxid#',
  'luxids': 'http://www.temis.com/luxid-schema#',
  'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
  'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
  'skos': 'http://www.w3.org/2004/02/skos/core#',
  'xsd': 'http://www.w3.org/2001/XMLSchema#'},
 '@graph': [{'@id': 'SRM0155',
   '@type': 'skos:Concept',
   'skos:broader': {'@id': 'SRM0192'},
   'skos:definition': {'@language': 'en',
    '@value': 'A characteristic of a population described by a statistic.'},
   'skos:prefLabel': {'@language': 'en', '@value': 'Population parameters'}},
  {'@id': 'SRM0232',
   '@type': 'skos:Concept',
   'skos:broader': {'@id': 'SRM0048'},
   'skos:definition': {'@language': 'en',
    '@value': 'The notion that language and action are indexical in the sense that their meaning depends on the context in which they arise.'},
   'skos:prefLabel': {'@language': 'en', '@value': 'Indexicality'}},
  {'@id': 'SRM0360',
   '@type': 'skos:Concept',
   's

In [38]:
labeldic,defdic,altlabeldic = {}, {}, {}
for i in range(len(methods['@graph'])):
    try:
        labeldic[methods['@graph'][i]["@id"]]=methods['@graph'][i]["skos:prefLabel"]['@value']
        defdic[methods['@graph'][i]["@id"]]=methods['@graph'][i]["skos:definition"]['@value']
        altlabeldic[methods['@graph'][i]["@id"]]=methods['@graph'][i]["skos:altLabel"]['@value']

    except:
        continue

print ("Tokenizing label")
(label_tokens_2,label_tokens_3) = tokenize_dataset(labeldic)





Tokenizing label


/Users/Ji/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:64: DeprecationWarning: generator 'ngrams' raised StopIteration


In [39]:
# labeldic

In [40]:
method = {}
for txt in train_data_tokens_2:
    method[txt] = []
    for i in label_tokens_2:
        for m in label_tokens_2[i]:
            if m in train_data_tokens_2[txt]:
                method[txt].append([m, i])

              


In [41]:
method3 = {}
for txt in train_data_tokens_3:
    method3[txt] = []
    for i in label_tokens_3:
        for m in label_tokens_3[i]:
            if m in train_data_tokens_3[txt]:
                method3[txt].append([m, i])  
                
  

In [42]:
# method3

In [43]:
result = []
for pub in method.keys():
    if method[pub] != []:
        for m in method[pub]:
            result.append({"publication_id": int(pub), "mention": labeldic[m[-1]], "score": 1.0})
for pub in method3.keys():
    if method3[pub] != []:
        for m in method3[pub]:
            result.append({"publication_id": int(pub), "mention": labeldic[m[-1]], "score": 1.0})
            
        

In [44]:
len(result)

75245

In [21]:
with open('methods.json', 'w') as f:
    json.dump(result, f)

In [23]:
json.dumps(result)

'[{"publication_id": 3268, "mention": "Covariance structure models", "score": 1.0}, {"publication_id": 3268, "mention": "Social network research", "score": 1.0}, {"publication_id": 3268, "mention": "Social reality", "score": 1.0}, {"publication_id": 3268, "mention": "Geographic information systems", "score": 1.0}, {"publication_id": 3268, "mention": "Process theory", "score": 1.0}, {"publication_id": 3268, "mention": "Social network analysis", "score": 1.0}, {"publication_id": 3268, "mention": "Social network analysis", "score": 1.0}, {"publication_id": 3268, "mention": "Structural analysis", "score": 1.0}, {"publication_id": 3268, "mention": "Social structures", "score": 1.0}, {"publication_id": 3268, "mention": "Teaching research methods", "score": 1.0}, {"publication_id": 3268, "mention": "Agent-based simulation", "score": 1.0}, {"publication_id": 3268, "mention": "Survey research", "score": 1.0}, {"publication_id": 3297, "mention": "Post-hoc analysis", "score": 1.0}, {"publication_